# Classifier Model Exercise 

In [69]:
#Importing pandas to read in the cvs
import pandas as pd

In [70]:
#Reading in the data
census_data = pd.read_csv('../DataSet/adult.csv')

In [71]:
#Printing the first 5
census_data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [72]:
#Getting the unique labels for the income column
census_data['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [73]:
#Creating a function to clean the data for income column
def fix_label(label):
    if label == '>50K':
        return 1
    else:
        return 0

In [74]:
#Cleaning up the data
census_data['income'] = census_data['income'].apply(fix_label)

In [75]:
#Creating the y_data
y_data = census_data['income']
y_data.head()

0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int64

In [76]:
#Creating the x_data
x_data = census_data.drop('income', axis=1)
x_data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


In [77]:
#import train_test_split from sklearn
from sklearn.model_selection import train_test_split

In [78]:
#Doing the train test split 
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3)

In [79]:
#Printing the columns out for the data 
census_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [80]:
#importing tensoflow
import tensorflow as tf

In [81]:
#Creating all the feature columns
age = tf.feature_column.numeric_column('age')
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=1000)
fnlwgt = tf.feature_column.numeric_column('fnlwgt')
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=1000)
education_num = tf.feature_column.numeric_column('education.num')
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital.status', hash_bucket_size=1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=1000)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=1000)
sex = tf.feature_column.categorical_column_with_vocabulary_list('sex', ['Female', 'Male'])
capital_gain = tf.feature_column.numeric_column('capital.gain')
capital_loss = tf.feature_column.numeric_column('capital.loss')
hours_per_week = tf.feature_column.numeric_column('hours.per.week')
native_country = tf.feature_column.categorical_column_with_hash_bucket('native.country', hash_bucket_size=1000)

In [82]:
#Putting all the feature columns all in one array
feat_cols = [age, workclass, fnlwgt, education, education_num, marital_status, occupation,
             relationship, race, sex, capital_gain, capital_loss, hours_per_week, native_country]

In [83]:
#Creating an input function for training the model
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=100, num_epochs=None,
                                                shuffle=True)

In [84]:
#Creating a Linear Classifier model
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpdll1ehdb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13cfa5ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [85]:
#Training the model
model.train(input_fn=input_func, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpdll1ehdb/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 185.176
INFO:tensorflow:loss = 148399.56, step = 101 (0.543 sec)
INFO:tensorflow:global_step/sec: 382.746
INFO:tensorflow:loss = 14865.119, step = 201 (0.261 sec)
INFO:tensorflow:global_step/sec: 383.089
INFO:tensorflow:loss = 97238.02, step = 301 (0.261 sec)
INFO:tensorflow:global_step/sec: 382.773
INFO:tensorflow:loss = 4887.95, step = 401 (0.260 sec)
INFO:tensorflow:global_step/sec: 386.112
INFO:tensorflow:loss = 11534.768, step = 501 (0.260 sec)
INFO:tensorflow:global_step/sec: 376.425
INFO:tensorflow:loss = 9745.656, step = 601 (0.266 sec)
INFO:tensorflow:gl

In [86]:
#Creating a prediction input function
pred_func = tf.estimator.inputs.pandas_input_fn(x=x_test, y=y_test, batch_size=len(x_test), shuffle=False)

In [87]:
#Getting prediction
pred_gen = model.predict(input_fn=pred_func)
predictions = list(pred_gen)
final_pred = [pred['class_ids'][0] for pred in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpdll1ehdb/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [88]:
#Importing classification_report to evalute the predictions
from sklearn.metrics import classification_report

In [89]:
print(classification_report(y_test, final_pred))

              precision    recall  f1-score   support

           0       0.79      0.99      0.87      7357
           1       0.81      0.18      0.30      2412

    accuracy                           0.79      9769
   macro avg       0.80      0.58      0.59      9769
weighted avg       0.79      0.79      0.73      9769

